![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [ ]:
# QuantBook Analysis Tool
# For more information see [https://www.quantconnect.com/docs/v2/our-platform/research/getting-started]
qb = QuantBook()
spy = qb.add_equity("SPY")
history = qb.history(qb.securities.keys(), 360*100, Resolution.DAILY)
history.index = history.index.droplevel(level=0)

In [ ]:

# Indicator Analysis
bbdf = qb.indicator(BollingerBands(30, 2), spy.symbol, 360*100, Resolution.DAILY)
bbdf.drop('standarddeviation', axis=1).plot()

rsi = qb.indicator(RelativeStrengthIndex(21), spy.symbol, 360*100, Resolution.DAILY)
rsi.rename(columns={'current': 'rsi'}, inplace=True)


adx = qb.indicator(AverageDirectionalIndex(21), spy.symbol, 360*100, Resolution.DAILY)
adx.rename(columns={'current': 'adx'}, inplace=True)

In [ ]:
"""RSI, BB, ADX"""
# —––––– 1. Align your data —–––––
# Assume you have:
#   price: a DataFrame indexed by datetime, with a 'close' column
#   rsi:   a DataFrame indexed by datetime, with a 'rsi' column
#   bbdf:  a DataFrame indexed by datetime, with a 'lowerband' column
#   adx:   a DataFrame indexed by datetime, with 'adx', 'positivedirectionalindex', 'negativedirectionalindex' columns
# Make sure all data frames share the same index (inner-join):
df = history[['open','high','low','close']].join(rsi[['rsi']], how='inner')\
                       .join(bbdf[['lowerband', 'upperband']], how='inner')\
                       .join(adx[['adx', 'positivedirectionalindex', 'negativedirectionalindex']], how='inner')

# —––––– 2. Identify signal dates —–––––
# RSI signal when RSI < 30
df['rsi_signal'] = df['rsi'] < 30

# BB signal when price closes below lower band
df['bb_signal'] = df['close'] < df['lowerband']

# Combined RSI and BB signal
df['rsi_bb_signal'] = (df['close'] < df['lowerband']) & (df['rsi'] < 30)

# ADX signal when ADX > 30 and +DI > -DI (bullish trend)
df['adx_signal'] = df['positivedirectionalindex'].pct_change() > 0 & (df['positivedirectionalindex'] > df['negativedirectionalindex']) & (df['adx'] > 25)

# ADX and RSI combined signal
df['adx_rsi_signal'] = df['adx_signal'] & (df['rsi'] < 30)

# ADX and BB combined signal
df['adx_bb_signal'] = df['adx_signal'] & (df['close'] < df['lowerband'])

# ADX, RSI, and BB combined signal
df['adx_rsi_bb_signal'] = df['adx_signal'] & (df['close'] < df['lowerband']) & (df['rsi'] < 30)

# —––––– 3. Compute forward returns for horizons 1–14 —–––––
# Create columns close_t+1 … close_t+14
for h in range(1, 15):
    df[f'close_t+{h}'] = df['close'].shift(-h)

# Compute forward returns: (close_{t+h}/close_t) - 1
for h in range(1, 15):
    df[f'return_{h}d'] = df[f'close_t+{h}'] / df['close'] - 1

# —––––– 4. Aggregate by signal type —–––––
# Define signal types and their labels
signal_types = [
    ('rsi_signal', 'RSI below 30'),
    ('bb_signal', 'Price < lower BB'),
    ('rsi_bb_signal', 'RSI < 30 and Price < lower BB'),
    ('adx_signal', 'ADX > 25'),
    ('adx_rsi_signal', 'ADX > 25 and RSI < 30'),
    ('adx_bb_signal', 'ADX > 25 and Price < lower BB'),
    ('adx_rsi_bb_signal', 'ADX > 25 and RSI < 30 and Price < lower BB')
]

# Calculate average returns for each signal type
returns_series = {}
for signal_column, label in signal_types:
    returns_series[label] = (
        df.loc[df[signal_column], [f'return_{h}d' for h in range(1, 15)]]
        .mean()
        .rename(label)
    )

# Calculate win rates and sample sizes efficiently
win_rate_dfs = {}
win_rate_series = {}

for signal_column, label in signal_types:
    # Calculate signal count
    total_signals = df[signal_column].sum()
    
    win_rates = []
    sample_sizes = []
    
    for h in range(1, 15):
        # Calculate positive returns
        positive_signals = df[df[signal_column] & (df[f'return_{h}d'] > 0)].shape[0]
        win_rate = positive_signals / total_signals if total_signals > 0 else 0
        
        win_rates.append(win_rate)
        sample_sizes.append(total_signals)
    
    # Create DataFrame with win rates and sample sizes
    idx = [f'return_{h}d' for h in range(1, 15)]
    win_rate_dfs[label] = pd.DataFrame({
        'Win Rate': win_rates,
        'Sample Size': sample_sizes
    }, index=idx)
    
    # Extract just the win rates for the summary table
    win_rate_series[f'{label} Win Rate'] = win_rate_dfs[label]['Win Rate'].rename(f'{label} Win Rate')

# —––––– 5. Combine into one summary table —–––––
# Interleave return metrics with win rates for each signal type
summary_data = []
for signal_column, label in signal_types:
    summary_data.append(returns_series[label])
    summary_data.append(win_rate_series[f'{label} Win Rate'])

summary = pd.concat(summary_data, axis=1)
summary.index.name = 'Days After Signal'
summary.columns.name = 'Signal Type'

print(summary)

# Print the detailed win rate DataFrames with sample sizes (not included in summary)
for label in [label for _, label in signal_types]:
    print(f"\n{label} Statistics:")
    print(win_rate_dfs[label])

In [ ]:
import interactive_charting as chart
fig = chart.equity_chart_with_signals(
    df, 
    start_date='2000-01-01', 
    end_date='2025-05-02',
    chart_type='candlestick',  # Use 'candlestick' or 'line'
    num_signals=4,
    signal_column_names=['rsi_bb_signal','adx_rsi_signal', 'adx_bb_signal', 'adx_rsi_bb_signal'],
    title='Candlestick Chart with Multiple Signals',
    marker_colors=['blue', 'purple', 'orange', 'red', 'green', 'yellow', 'pink'],
    marker_symbols=['triangle-down', 'triangle-up', 'circle'],
    candlestick_increasing_color='limegreen',
    candlestick_decreasing_color='crimson'
)

# Show the figure
fig.show()